# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [1]:
from pathlib import Path
import sys
sys.path.append('..')
import argparse
import shutil
import os
import logging
from textblob import TextBlob

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import *

from finbert.finbert import *
import finbert.utils as tools
from pprint import pprint
from sklearn.metrics import classification_report
%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

In [2]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [3]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [4]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = BertForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 64,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True )

10/30/2019 09:42:39 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /home/ubuntu/finbert_test/finbert/models/language_model/finbertTRC2 from cache at /home/ubuntu/finbert_test/finbert/models/language_model/finbertTRC2
10/30/2019 09:42:39 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

10/30/2019 09:42:42 - INFO - pytorch_pretrained_bert.modeling -   Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
10/30/2019 09:42:42 - INFO - pytorch_pretrained_bert.modeling -   Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.tr

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [5]:
finbert = FinBert(config)

In [6]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

10/30/2019 09:42:45 - INFO - finbert.finbert -   device: cuda n_gpu: 8, distributed training: False, 16-bits training: False
10/30/2019 09:42:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Fine-tune the model

In [7]:
# Get the training examples
train_data = finbert.get_data('train')

In [8]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [ ]:
# This is for fine-tuning a subset of the model.

freeze = 11

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [9]:
trained_model = finbert.train(train_examples = train_data, model = model)

10/30/2019 09:42:52 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:42:52 - INFO - finbert.utils -   guid: train-1
10/30/2019 09:42:52 - INFO - finbert.utils -   tokens: [CLS] weber convinced his friend ray os ##trom , who owned a lake street sporting goods store , to sell the lure ##s . [SEP]
10/30/2019 09:42:52 - INFO - finbert.utils -   input_ids: 101 13351 6427 2010 2767 4097 9808 13887 1010 2040 3079 1037 2697 2395 7419 5350 3573 1010 2000 5271 1996 17256 2015 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:42:52 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:42:52 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:42:52 - INFO - finbert.utils -   label: neutr

10/30/2019 09:43:20 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:43:20 - INFO - finbert.utils -   guid: validation-1
10/30/2019 09:43:20 - INFO - finbert.utils -   tokens: [CLS] the fair value of the company ' s investment properties grew to eu ##r 2 . 80 ##3 billion at the end of march 2009 from eu ##r 2 . 69 ##1 million a year ago . [SEP]
10/30/2019 09:43:20 - INFO - finbert.utils -   input_ids: 101 1996 4189 3643 1997 1996 2194 1005 1055 5211 5144 3473 2000 7327 2099 1016 1012 3770 2509 4551 2012 1996 2203 1997 2233 2268 2013 7327 2099 1016 1012 6353 2487 2454 1037 2095 3283 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:43:20 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:43:20 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

10/30/2019 09:43:21 - INFO - finbert.finbert -   ***** Loading data *****
10/30/2019 09:43:21 - INFO - finbert.finbert -     Num examples = 776
10/30/2019 09:43:21 - INFO - finbert.finbert -     Batch size = 32
10/30/2019 09:43:21 - INFO - finbert.finbert -     Num steps = 96



Validation losses: [0.9508100390434265]
No best model found


Epoch:  25%|██▌       | 1/4 [00:33<01:39, 33.33s/it]

...

Epoch:  75%|███████▌  | 3/4 [02:32<00:47, 47.51s/it]

10/30/2019 09:46:36 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
10/30/2019 09:46:36 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
10/30/2019 09:46:37 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
10/30/2019 09:46:37 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:46:37 - INFO - finbert.utils -   guid: validation-1
10/30/2019 09:46:37 - INFO - finbert.utils -   tokens: [CLS] the fair value of the company ' s investment properties grew to eu ##r 2 . 80 ##3 billion at the end of march 2009 from eu ##r 2 . 69 ##1 million a year ago . [SEP]
10/30/2019 09:46:37 - INFO - finbert

10/30/2019 09:46:38 - INFO - finbert.finbert -   ***** Loading data *****
10/30/2019 09:46:38 - INFO - finbert.finbert -     Num examples = 776
10/30/2019 09:46:38 - INFO - finbert.finbert -     Batch size = 32
10/30/2019 09:46:38 - INFO - finbert.finbert -     Num steps = 96


Epoch: 100%|██████████| 4/4 [03:50<00:00, 57.53s/it]


Validation losses: [0.9508100390434265, 0.4066435068845749, 0.357919294834137, 0.3599339699745178]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [10]:
test_data = finbert.get_data('test')

In [11]:
results = finbert.evaluate(examples=test_data, model=trained_model)

10/30/2019 09:48:03 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:48:03 - INFO - finbert.utils -   guid: test-1
10/30/2019 09:48:03 - INFO - finbert.utils -   tokens: [CLS] alt ##imo and tel ##ias ##one ##ra said yesterday that us ##man ##ov would be welcome to join the new company . [SEP]
10/30/2019 09:48:03 - INFO - finbert.utils -   input_ids: 101 12456 16339 1998 10093 7951 5643 2527 2056 7483 2008 2149 2386 4492 2052 2022 6160 2000 3693 1996 2047 2194 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:48:03 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:48:03 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/30/2019 09:48:03 - INFO - finbert.utils -   label: positive (

### Prepare the classification report

In [12]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [13]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [14]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.35
Accuracy:0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.86      0.78       238
           1       0.78      0.90      0.84       125
           2       0.94      0.83      0.88       606

    accuracy                           0.85       969
   macro avg       0.81      0.86      0.83       969
weighted avg       0.86      0.85      0.85       969



### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [15]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [16]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = BertForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

10/30/2019 09:48:19 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /home/ubuntu/finbert_test/finbert/models/classifier_model/finbert-sentiment from cache at /home/ubuntu/finbert_test/finbert/models/classifier_model/finbert-sentiment
10/30/2019 09:48:19 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [17]:
result = predict(text,model)

10/30/2019 09:48:24 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
10/30/2019 09:48:24 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:48:24 - INFO - finbert.utils -   guid: 0
10/30/2019 09:48:24 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
10/30/2019 09:48:24 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [18]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.022088699, 0.91169304, 0.06621831]",negative,-0.889604,0.051746
1,The news rapidly infected financial markets.,"[0.17535701, 0.47266006, 0.35198295]",negative,-0.297303,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.016599067, 0.97293335, 0.010467538]",negative,-0.956334,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.37171096, 0.5898656, 0.038423426]",negative,-0.218155,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.018420218, 0.9672908, 0.014289]",negative,-0.948871,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.0890439, 0.8222241, 0.088732034]",negative,-0.733180,0.000000


In [19]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.67.


Here is another example

In [20]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [21]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

10/30/2019 09:48:41 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
10/30/2019 09:48:41 - INFO - finbert.utils -   *** Example ***
10/30/2019 09:48:41 - INFO - finbert.utils -   guid: 0
10/30/2019 09:48:41 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
10/30/2019 09:48:41 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
10/

In [22]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.93742305, 0.022251064, 0.040325828]",positive,0.915172,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.037236165, 0.007026143, 0.95573765]",neutral,0.030210,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.33458936, 0.0103126755, 0.65509796]",neutral,0.324277,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.91061854, 0.018312845, 0.07106857]",positive,0.892306,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.60809374, 0.040814754, 0.3510915]",positive,0.567279,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.19234878, 0.008732705, 0.79891855]",neutral,0.183616,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.78942287, 0.015099782, 0.19547734]",positive,0.774323,0.000000


In [23]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.51.
